In [ ]:
import socket, threading, time, Utils as ut, Statistics as st

sf, sn, sw = 0, 0, 4  # sf = send frame, sn = next frame, sw = sliding window (0-based index)
text, data_queue = [], [None] * sw
flag, flag2, length = False, False, 0

sender = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sender.bind(st.ADDR)

def send(conn):
    global sn, sf, text, flag
    
    while sn - sf < sw and text:
        data = text.pop(0)
        frame = ut.noisychannel(ut.makeFrame(sn, data))
        print(f"[SENDING] Frame: {sn}, {frame}")
        conn.send(frame.encode())
        data_queue[sn % sw] = data
        sn += 1
    
    if not text:
        print("[FINISHED] All output read.")
        flag = True

def recv_Ack(conn):
    global sf, sn, sw, data_queue, flag2

    while True:
        ack_frame = conn.recv(20).decode()
        ack_no, _, data, _ = ut.receiveFrame(ack_frame)

        if data == '00001111' and sf <= ack_no <= sn:  # NAK
            print(f"[NAK RECEIVED] NAK {ack_no}")
            resend_frame(conn, ack_no)

        elif data == '11110000' and sf <= ack_no <= sn:  # ACK
            print(f"[ACK RECEIVED] ACK {ack_no}")
            while sf <= ack_no:
                data_queue[sf % sw] = None
                sf += 1
            if ack_no == 7:  # Termination check
                break

def resend_frame(conn, ack_no):
    frame = ut.makeFrame(ack_no, data_queue[ack_no % sw])
    print(f"[RESENDING] Frame: {ack_no}, {frame}")
    conn.send(frame.encode())

def start():
    global text, length
    
    sender.listen()
    print(f"[LISTENING] Server is listening on {st.HOST_IP}")
    conn, addr = sender.accept()
    print(f"[CONNECTED] Process Id: {addr}")

    with open("data.txt", "r") as f:
        text = [line.strip() for line in f.readlines()]
        length = len(text)

    threading.Thread(target=send, args=(conn,)).start()
    threading.Thread(target=recv_Ack, args=(conn,)).start()

    print("[CLOSING] Closing sender...")
    conn.close()

start()
